In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import quote
from pprint import pprint
import random
import csv
import re
from random import randint
from time import sleep
from datetime import datetime


class GoogleSearchScrapper(object):
    def __init__(self):
        super().__init__()
        user_agent_list = [
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
        ]
        user_agent = user_agent_list[random.randrange(len(user_agent_list))]
        self.headers = {
            'User-Agent': user_agent,
            'Host': 'www.google.com',
            'Referer': 'https://www.google.com/'
        }
        print("[INFO] User Agent:",user_agent)

    def __get_source(self, url: str) -> requests.Response:
        return requests.get(url, headers=self.headers)

    def search(self, query: str, num_of_pages: int) -> list:
        results = []
        filename = "results_"+datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")+".csv"
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["query","title", "url", "description","bold","biased_sentence"])
            for page in range(0,int(num_of_pages)*10,10):
                # Sleeping for X seconds
                sleep_time = randint(1,10)
                print(f"[INFO] Sleeping: {sleep_time} Seconds")
                sleep(sleep_time)
                
                # Printing URL and Page Number 
                url = 'https://www.google.com/search?q=%s' % quote(query)+'&start='+str(page)+'&filter=0'
                page_number = int(page/10)
                print(f"[INFO] URL: {url} Page #",page_number+1)
                
                response = self.__get_source(url)
                soup = BeautifulSoup(response.text, 'html.parser')
                result_containers = soup.findAll('div', class_='tF2Cxc')

                for container in result_containers:
                    try:
                        title = container.find('h3').text
                        url = container.find('a')['href']
                        description = container.find('span', class_='aCOpRe').text
                        bold = container.find('span', class_='aCOpRe').find('em').text
                        bias = re.findall(r"([^.]*?"+bold+"[^.]*\.)",description)
                    except:
                        continue
                    writer.writerow([query,title, url, description,bold,bias])
                    results.append({
                        'query': query,
                        'title': title,
                        'url': url,
                        'description': description,
                        'bold': bold,
                        'biased_sentence': bias
                    })
        print(f"[INFO] {len(results)} Results Saved to CSV File [{filename}]")
        return results

if __name__ == '__main__':
    GoogleSearchScrapper().search(input('[INFO] Query: '),input('[INFO] Number of Pages: '))

[INFO] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0
[INFO] Query: "A great teacher * his"
[INFO] Number of Pages: 3
[INFO] Sleeping: 2 Seconds
[INFO] URL: https://www.google.com/search?q=%22A%20great%20teacher%20%2A%20his%22&start=0&filter=0 Page # 1
[INFO] Sleeping: 9 Seconds
[INFO] URL: https://www.google.com/search?q=%22A%20great%20teacher%20%2A%20his%22&start=10&filter=0 Page # 2
[INFO] Sleeping: 6 Seconds
[INFO] URL: https://www.google.com/search?q=%22A%20great%20teacher%20%2A%20his%22&start=20&filter=0 Page # 3
[INFO] 29 Results Saved to CSV File [results_2021_02_27-07_30_43_PM.csv]
